In [64]:
import netCDF4
import itertools

from tqdm import tqdm

Nx = 512
Ny = 512
Nz = 601
Na = 5
Ns = 25

In [1]:
mu_id = ['10', '8', '6', '4', '2']

T = np.zeros((Na, Nz, Ny * Nx))
t = np.zeros((Na, Nz, Ny * Nx))

T_ssd = np.zeros((Na, Nz, Ns))
t_ssd = np.zeros((Na, Nz, Ns))

T_300 = np.zeros((Na, Nz, Ns))
t_300 = np.zeros((Na, Nz, Ns))

for i in range(Na):
    
    T[i, :, :] = np.array(netCDF4.Dataset('./ssd/222809/' + mu_id[i] + '/T_onTau.222809.nc.1')['T']).reshape(Nz, Ny * Nx)
    t[i, :, :] = np.array(netCDF4.Dataset('./ssd/222809/' + mu_id[i] + '/taugrid.222809.nc.1')['tau']).reshape(Nz, Ny * Nx)
    
    sample = np.random.choice(Nx * Ny, Ns, replace = False)
    
    T_ssd[i, :, :] = T[i, :, sample].T
    t_ssd[i, :, :] = t[i, :, sample].T

    T[i, :, :] = np.array(netCDF4.Dataset('./300G/627321/' + mu_id[i] + '/T_onTau.627321.nc.1')['T']).reshape(Nz, Ny * Nx)
    t[i, :, :] = np.array(netCDF4.Dataset('./300G/627321/' + mu_id[i] + '/taugrid.627321.nc.1')['tau']).reshape(Nz, Ny * Nx)
    
    sample = np.random.choice(Nx * Ny, Ns, replace = False)
    
    T_300[i, :, :] = T[i, :, sample].T
    t_300[i, :, :] = t[i, :, sample].T

np.savez('intro_img_2', t_ssd = t_ssd, t_300 = t_300, T_ssd = T_ssd, T_300 = T_300)

ModuleNotFoundError: No module named 'netCDF4'

In [65]:
t_ssd = np.load('intro_img_2.npz')['t_ssd']
t_300 = np.load('intro_img_2.npz')['t_300']
T_ssd = np.load('intro_img_2.npz')['T_ssd']
T_300 = np.load('intro_img_2.npz')['T_300']

In [98]:
import itertools

from scipy import interpolate

def find_interpol_range(tau, Nz, Ns):

    thick = 1e+20
    thinn = 1e-20

    for k in range(Ns):

        opt_thick_end = tau[Nz - 1, k]
        opt_thinn_end = tau[0, k]

        if opt_thinn_end > thinn: thinn = opt_thinn_end
        
        if opt_thick_end < thick: thick = opt_thick_end
    
    return np.log10(thinn), np.log10(thick)

def find_shortest_delta(tau, Nz, Ns):
    
    delta = 20
    
    for j, k in itertools.product(range(Nz - 1), range(Ns)):
        
        d = np.abs(np.log10(tau[j + 1, k]) - np.log10(tau[j, k]))
        
        if d < delta and d > 0.0001: delta = d
            
    return delta

def const_interpol_grid(thin, thick, delta):
    
    Ng = np.int(np.floor(np.abs(thick - thin) / delta))
    
    grid = np.zeros(Ng)
    
    grid[0] = thin
    
    for i in range(Ng - 1): grid[i + 1] = grid[i] + delta
    
    return grid

def interpol_and_average(tau, T, igrid, Ns):
    
    T_mean = np.zeros(len(igrid))
    
    for k in range(Ns):
    
        f = interpolate.interp1d(np.log10(tau), T)
        
        Ti = f(igrid)
        
        T_mean += Ti
        
    T_mean /= Ns
    
    return T_mean

grid_ssd = np.ones((Na, 110000)) * np.nan
grid_300 = np.ones((Na, 110000)) * np.nan

Ti_ssd = np.ones((Na, 110000)) * np.nan
Ti_300 = np.ones((Na, 110000)) * np.nan

for i in range(Na):
    
    thin, thick = find_interpol_range(t_ssd[i, :, :], Nz, Ns)
    delta =       find_shortest_delta(t_ssd[i, :, :], Nz, Ns)
    grid =        const_interpol_grid(thin, thick, delta)
    T_mean =      interpol_and_average(t_ssd[i, :, k], T_ssd[i, :, k], grid, Ns)
    
    grid_ssd[i, : len(grid)] = 10.0**grid    
    Ti_ssd[i, : len(T_mean)] = T_mean
    
    thin, thick = find_interpol_range(t_300[i, :, :], Nz, Ns)
    delta =       find_shortest_delta(t_300[i, :, :], Nz, Ns)
    grid =        const_interpol_grid(thin, thick, delta)
    T_mean =      interpol_and_average(t_300[i, :, k], T_300[i, :, k], grid, Ns)
    
    grid_300[i, : len(grid)] = 10.0**grid
    Ti_300[i, : len(T_mean)] = T_mean

In [9]:
import itertools

for i, k, j in itertools.product(range(Na), range(Ns), range(Nz - 1)):
            
    delta = np.abs(np.log10(t_ssd[i, j, k]) - np.log10(t_ssd[i, j + 1, k]))
        
    if np.abs(delta - 0.0001) < 1e-6: T_ssd[i, j, k] = np.nan
    
    delta = np.abs(np.log10(t_300[i, j, k]) - np.log10(t_300[i, j + 1, k]))

    if np.abs(delta - 0.0001) < 1e-6: T_300[i, j, k] = np.nan
            
np.savez('intro_img_2_nans', t_ssd = t_ssd, t_300 = t_300, T_ssd = T_ssd, T_300 = T_300)
            
t_ssd = np.load('intro_img_2_nans.npz')['t_ssd']
t_300 = np.load('intro_img_2_nans.npz')['t_300']
T_ssd = np.load('intro_img_2_nans.npz')['T_ssd']
T_300 = np.load('intro_img_2_nans.npz')['T_300']

In [101]:
import matplotlib.pyplot as plt

import matplotlib.gridspec as gridspec

props = dict(boxstyle = 'round', facecolor = 'wheat', alpha = 0.5)

plt.rcParams.update({'font.size': 8})
plt.rcParams["font.family"] = 'Times New Roman'

mus = ['1.0', '0.8', '0.6', '0.4', '0.2']

fig = plt.figure(dpi = 300)

fig.set_size_inches(10, 11.25, forward = True)

fig.tight_layout()

gs = gridspec.GridSpec(5, 3, width_ratios = [1, 1, 1], hspace = 0.2, wspace = 0.2)

for i in range(5):

    ssd = plt.subplot(gs[i, 0])
    mag = plt.subplot(gs[i, 1])
        
    com = plt.subplot(gs[i, 2])
    
    for k in range(Ns):
        
        ssd.plot(t_ssd[i, :, k], T_ssd[i, :, k], color = 'gray')
        mag.plot(t_300[i, :, k], T_300[i, :, k], color = 'gray')
        
    ssd.plot(grid_ssd[i, :], Ti_ssd[i, :], color = 'k')
    mag.plot(grid_300[i, :], Ti_300[i, :], color = 'r')
        
#        if i == 0:
            
#            ssd.axvline(x = thick, color = 'r', linestyle = '--')
#            ssd.axvline(x = thinn, color = 'r', linestyle = '--')
        
    ssd.set_xscale('log')
    
    ssd.set_xlim(3e+1, 2e-12)
    ssd.set_ylim(2000, 10500)
    
    ssd.set_ylabel('Temperature, K')
    
    if i == 0: ssd.set_title('SSD')
    
    mag.set_xscale('log')
    
    mag.set_xlim(ssd.get_xlim())
    mag.set_ylim(ssd.get_ylim())
    
    if i == 0: mag.set_title('300G')
    
    if i == 4: ssd.set_xlabel(r'$\tau_\mathrm{Ross}$')
    if i == 4: mag.set_xlabel(r'$\tau_\mathrm{Ross}$')
    if i == 4: com.set_xlabel(r'$\tau_\mathrm{Ross}$')
    
    com.set_ylabel(r'$\mu =$' + mus[i])
    com.yaxis.set_label_position("right")

plt.savefig('intro_img_2.pdf', bbox_inches = 'tight')

plt.close('all')